In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage


class Memory:
    def __init__(self):
        self.memory = ConversationBufferMemory(return_messages=True)

    def save_memory(self, input_text, output_text):
        self.memory.save_context(
            {"input": input_text},
            {"output": output_text}
        )

    def load_memory_variables(self):
        return self.memory.load_memory_variables({})

    def get_chat_history(self):
        """채팅 히스토리를 문자열로 반환"""
        messages = self.memory.chat_memory.messages
        history = []
        for message in messages:
            if isinstance(message, HumanMessage):
                history.append(f"Human: {message.content}")
            elif isinstance(message, AIMessage):
                history.append(f"AI: {message.content}")
        return "\n".join(history)


class Document:
    def __init__(self, file_path):
        self.loader = UnstructuredFileLoader(file_path)
        self.splitter = CharacterTextSplitter.from_tiktoken_encoder(
            separator="\n",
            chunk_size=600,
            chunk_overlap=100,
        )
        self.docs = self.loader.load_and_split(text_splitter=self.splitter)

    def get_documents(self):
        return self.docs


class StuffDocumentsChain:
    def __init__(self, docs, memory):
        self.llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            streaming=True,
            verbose=True
        )
        self.embeddings = OpenAIEmbeddings()
        self.cache_dir = LocalFileStore('./.cache/')
        self.cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
            self.embeddings, self.cache_dir
        )
        self.vectorstore = FAISS.from_documents(docs, self.cached_embeddings)
        self.retriever = self.vectorstore.as_retriever()
        self.memory = memory

    def create_prompt_with_memory(self, question, context, chat_history=""):
        prompt_template = """You are a helpful assistant. Answer questions using only the following context and conversation history.
          If you don't know the answer just say you don't know, don't make it up.

          Previous conversation:
          {chat_history}

          Context:
          {context}

          Question: {question}

          Answer:
        """

        return prompt_template.format(
            chat_history=chat_history,
            context=context,
            question=question
        )

    def stuff_documents(self, docs):
        return "\n\n".join([doc.page_content for doc in docs])

    def invoke(self, question):
        relevant_docs = self.retriever.get_relevant_documents(question)
        context = self.stuff_documents(relevant_docs)
        chat_history = self.memory.get_chat_history()
        full_prompt = self.create_prompt_with_memory(
            question, context, chat_history)

        response = self.llm.invoke([HumanMessage(content=full_prompt)])
        self.memory.save_memory(question, response.content)

        return response


if __name__ == "__main__":
    doc = Document('./files/chapter_three.txt')
    documents = doc.get_documents()
    memory = Memory()
    stuff_chain = StuffDocumentsChain(documents, memory)

    questions = [
        "Is Aaronson guilty?",
        "What message did he write in the table?",
        "Who is Julia?"
    ]

    # 각 질문 실행 및 결과 출력
    for i, question in enumerate(questions, 1):
        print(f"\n{'=' * 50}")
        print(f"Question {i}: {question}")
        print(f"{'=' * 50}")

        try:
            result = stuff_chain.invoke(question)
            print("Answer:")
            print(result.content)

        except Exception as e:
            print(f"Error Occured: {e}")

    print(f"\n{'=' * 50}")
    print("Final Chat History:")
    print(f"{'=' * 50}")
    print(memory.get_chat_history())


Question 1: Is Aaronson guilty?
Answer:
He was guilty of the crimes he was charged with.

Question 2: What message did he write in the table?
Answer:
He wrote "2+2=5" in the dust on the table.

Question 3: Who is Julia?
Answer:
Julia is a character mentioned in the context, who Winston loves and feels a deep connection to, especially during moments of emotional turmoil.

Final Chat History:
Human: Is Aaronson guilty?
AI: He was guilty of the crimes he was charged with.
Human: What message did he write in the table?
AI: He wrote "2+2=5" in the dust on the table.
Human: Who is Julia?
AI: Julia is a character mentioned in the context, who Winston loves and feels a deep connection to, especially during moments of emotional turmoil.
